# Urban Traffic Bonn
Simulate the urban traffic from the city of Bonn using [ArcGIS API for Python](https://developers.arcgis.com/python/).

In [ ]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

## Required Python modules
You need to install ArcGIS API for Python, please follow the [guide](https://developers.arcgis.com/python/guide/anaconda/).

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from datetime import date, datetime
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from geourban.services import aggregate, query, simulations, top
from geourban.types import GridType, VehicleType

## Connect to ArcGIS Online

In [ ]:
gis = GIS()

## Create a map showing the city of Bonn

In [ ]:
bonn_map = gis.map('Bonn, Germany')
bonn_map

# Connect to geourban services
The host parameter must target the specific host like "geoprotests.p.rapidapi.com". Furthermore, the factory directly access `os.environ['x_rapidapi_key']` and uses the specified API key as a header parameter. Otherwise, `georapid.factory.EnvironmentClientFactory.create_client_with_host()` will raise a `ValueError`.

Please, check out the [RapidAPI Account Creation and Management Guide](https://docs.rapidapi.com/docs/account-creation-and-settings).

In [ ]:
client = EnvironmentClientFactory.create_client_with_host('geourban.p.rapidapi.com')

# List the available simulations

In [ ]:
urban_simulations = simulations(client)
urban_simulations

## Request the top 5 accumulated car traffic grid cells
We request these hotspots for the city of Bonn by using the urban region code `DEA22`, the simulation date `2023-08-24`, the vehicle type `Car`, and the grid type `agent`. The returned GeoJSON features represents the grid cells with the highest car throughput.

In [ ]:
bonn_region_code = 'DEA22'
simulation_date = date(2023, 8, 24)
vehicle_type = VehicleType.CAR
grid_type = GridType.AGENT
limit = 5
top_traffic_grid_cells = top(client, bonn_region_code, simulation_date, vehicle_type, grid_type, limit=limit)
top_traffic_grid_cells

## Convert the returned GeoJSON result into a FeatureSet
The FeatureSet offers direct access to a spatially enabled dataframe. We can easily inspect the time frames (`start_time` - `end_time`) and the number of car vehicles `agent_count`.

In [ ]:
bonn_top_traffic_fset = FeatureSet.from_geojson(top_traffic_grid_cells)
top_traffic_sdf = bonn_top_traffic_fset.sdf
top_traffic_sdf

## Map the traffic grid cells

In [ ]:
top_traffic_sdf.spatial.plot(bonn_map)
bonn_map